## 41. Ch 08. Recurrent Neural Networks - 01. RNN 소개

In [ ]:
기존에 다뤄왔던 뉴럴 네트워크와는 다른 구조
-Previous Methods
예. Tabular Data, Image
x -> Model -> y
y = f(x;세타)

-Recurrent Neural Networks
예. Sequential Data, Time-series
xt -> 모델 - ht -> xt
출력이 입력값으로 들어간다
ht = f(xt, ht-1;세타)
->y
입력 받을 때 이전 타임스텝의 출력과 현재 타임 스텝의 입력을 동시에 받는다
따로 y라는 게 존재하지 않는다고 생각하는 게 좋음

<Sequential Data vs Time Series>
-Time-stamp의 유무에 따른 차이
	-시퀀셜 데이터는 데이터의 순서 정보가 매우 중요함
	예. 텍스트 문장: 단어의 순서
	-추가로 시계열 데이터는 해당 데이터가 발생한 시각 정보가 매우 중요함
	예. 주식 데이터: 가격의 순서 및 발생 시점
-Time Series Data(시계열 데이터)
주식 데이터(거래 시점)
센서 데이터
-Sequential Data
텍스트
(샘플링 주기가 일정한) 영상/음성

RNN은 타임시리즈 데이터를 위한 아키텍처라고 볼 수 없다.
타임스탬프를 바로 받아들일 수 있는 아키텍처는 아니고 시퀀셜 데이터를 위한 아케틱처에 오히려 맞는 편이긴 함
우리가 시계열 데이터를 모은 다음에 우리가 인터벌을 일정하게 바꿔준다거나 샘플링 레이트를 일정하게 바꿔주는 전처리를 거치게 돼서
시퀀셜 데이터로 만들 수 있게 된다면 RNN에 넣을 수 있다.

기존의 FC모듈, FC레이어나 CNN같은 경우 애초 순서 정보, 이전 데이터에 기반한 현재 데이터 처리가 불가능한 반면에,
RNN은 순서 정보나 이전 데이터에 기반한 현재 데이터의 처리가 가능하다는 특징을 갖고 있다.

<RNN, 더 이상 어려워하지 말자!>
-시간(순서) 개념이 추가되니 어려움을 느낄 수 있음
Generation task가 아니면 사실 시간 개념을 구현에 넣을 필요가 없음
일단 입/출력 텐서의 모양만 알아도 이미 반은 해결
-Back-propagation이 어렵다?
수식을 몰라도 기본 원리만 기억하자
-스텝바이스텝으로 접근하면 많은 variation에 대해서도 해결됨

## 42. Ch 08. Recurrent Neural Networks - 02. RNN Step-by-Step 들여다보기

In [ ]:
현재 타임스텝의 입력과 이전 타임스텝의 출력을 입력으로 함께 받는다.
y라는 것이 따로 없음
계속 뱅글뱅글 돈다.
tanh를 논리니어액티베이션 펑션으로 쓴다.

yt hat = ht = f(xt, ht-1;세타)
=tanh(Wih xt + bis) + (Whh ht-1 + bhh)
where 세타 = {Wih, bih, Whh, bhh}
= tanh(W[xt, ht-1] # 바이어스는 보통 많이 생략함

세타는 역전파되기 전까지 모두 동일
세타는 x와 h가 들어오는 거에 대해서 시간에 관계 없이 항상 잘 동작해야 한다고 할 수 있다

<How it works>
L(세타) = 시그마(t=1, T) ||yt hat - yt||

<Zoom In : Input Tensor>
|xt| = (batch_size, 1, input_size) *n -> |X| = (batch_size, n, input_size) where X = {x1,x2,…,xn}
단어 -> 문장

이미지는 (N, C, H, W)의 4차원 텐서 사용
문장은 (N, Length, Dimension)의 3차원 텐서 사용

생성 태스크가 아닌 경우에는 오른쪽처럼 한번에 데이터를 넣어 준다.

<Zoom In : Hidden Tensor>
히든 스테이트 텐서
출력이라고도 볼 수 있다

|ht| = (batch_size, hidden_size) *n -> |h1:n| = (Batch_size, n, hidden_size) where h1:n = [h1;h2;…;hn]
입력에 전체 시간이 한 번에 들어가면 한 번에 다 출력으로 나온다

<Multi-layered RNN>
앞전까지는 싱글 레이어 RNN
어떻게 구현할지 고민할 필요 없다. 파이토치 파라미터(number of layers)로 지정해주면 된다.
마지막 레이어의 히든 스테이트가 y hat이 된다.
세타는 타임스텝에 따라선 모두 같지만 층에 대해선 다르다.
각 층 마다 다른 파라미터

<Zoom In : Output Tensor>
|h1:n| = (batch_size, n, hidden_size)

<Zoom In : Hidden State Tensor>
|ht| = (#layers, batch_size, hidden_size)

<Bidirectional Multi-layered RNN>
RNN을 두 방향으로도 만들 수 있다. 정방향, 역방향.
통째로 입력해주고 통째로 출력을 뱉는 문제 형태에서는 가장 많이 사용.

<Zoom In : Output Tensor>
마지막 레이어의 정방향의 각 타임스텝별 히든스테이트들과 마지막 레이어의 역방향의 각 타임스텝별 히든스테이트들이 모두 모이게 된다.
|h1:n| = (batch_size, n, hidden_size X #directions), directions=2

<Zoom In : Hidden State Tensor>
|ht| = (#directions X #layers, batch_size, hidden_size), directions=2
t라고 해놨기 때문에 시간 개념이 안 들어 있음. 이 텐서 안에는
엄청 중요하지는 않다.
바이디렉셔널 멀티레이어 RNN을 하게 되면 전체 타임스텝으로 입력, 전체 타임스텝으로 출력되기 때문에
다음 입력 받기 위한 작업들을 잘 하지 않는 경우가 대부분.
그래도 구조는 알아야.

<You can refer documents, anytime>
파이토치 다큐먼트 잘 돼 있다.
Input shape
Output shape

<요약>
-Single layer RNN에서 hidden state는 곧 output이다.
-Multi-layered RNN에서
	-Output은 마지막 레이어의 모든 타임스텝의 히든스테이트이다.
	-히든 스테이트는 마지막 타임 스텝의 모든 레이어의 히든 스테이트이다.
-Bi-directional RNN에서
	-Output은 hidden state의 2배가 된다
	-히든 스테이트는 레이어의 개수가 2배가 된다
기본 원리만 잘 이해하고 있어도 어렵지 않게 사용할 수 있다.

## 43. Ch 08. Recurrent Neural Networks - 03. RNN 활용 사례

In [ ]:
<Applications>
-Many(입력) to One(출력)/Text Classification(문장 받은 다음 어떤 클래스에 속하는지 결과값으로)
-One to Many/NLG, Machine Translation
-Many to Many/POS Tagging(형태소분석), MRC(Machine Reading Comprehesion)

<Two Approaches>
1.Non-autoregressive(Non-generative)
-현재 상태가 앞/뒤 상태를 통해 정해지는 경우
	-예.Part of Speech(POS) Tagging, Text Classification
-Bidirectional RNN 사용 권장(문장의 처음과 끝을 다 받은 상태이므로)
문장 전체를 다 받거나 시퀀스를 다 받는 경우
2.Autoregressive(Generative)
-현재 상태가 과거 상태에 의존해 정해지는 경우
이전 타임스텝의 정보를 가지고 현재가 정해진다.
	-예. NLG, 기계 번역
	이전에 단어가 있고 이에 따라 현재에는 어떤 단어를 뱉어야 하는지를 고민하는 태스크
-One-to-Many case 해당
-Bidirectional RNN 사용 불가!
미래의 타임스텝에 대한 입력은 아직 주어지지 않음.
예. 오전 주식 데이터만 주고 오후 것을 예측하라고 하면 Bidirectional RNN 쓸 수 없다.
예. 문장도 마찬가지

<Many to Many>
예. POS Tagging 형태소 분석

<Many to One>
예. 텍스트 분류

<One to Many>
가장 어려운 태스크
예. NLG

<One to Many(시퀀스투시퀀스)>
예. 기계번역
Encoder -> Decoder
Many to one / One to Many

시퀀스투시퀀스
시퀀스를 한 점으로, 점을 다시 시퀀스로 바꿔내는 형태

-> 인코더의 마지막 히든스테이트를 디코더의 이니셜 스테이트로 넣어주면 이를 바탕으로 문장 생성

## 44. Ch 08. Recurrent Neural Networks - 04. RNN에서의 Back-propagation (BPTT)

In [ ]:
Back Propagation Through Time(BPTT)
<Back Propagation with Multiple Forwarding Paths>

<Back Propagation in RNN>
-Many to One
타입스텝마다의 세타로 미분해준 그레디언트를 모두 더한다
타임스텝에 따라 그레디언트 크기가 바뀜
타임스텝이 길어질수록 그레디언트 크기가 커짐
너무 긴 시퀀스의 경우에는 그레디언트가 폭발하기도(그레디언트 익스플로딩)

-Many to Many
한 타임 스텝 안에서도 두 개의 피드포워드 경로가 존재
이를 다 더해야 함.

-Many to Many with Multi-layered RNN
더 다양하게 다 더해진다

<TanH in Vanilla RNN> 여러개의 깊은 레이어가 쌓인 형태
마디마다 tanH가 쓰이고 있다
깊은 네트워크에서 발생하는 기울기 소멸이 발생할 수밖에 없다.
tanH는 -무한대에서는 -1, +무한대에서는 +1
tanH가 0일 때 기울기의 값은 1이 된다
tanH에 들어가는 값이 0일 때를 제외하고는 모든 그레디언트가 1보다 작게 된다.
계속해서 역전파가 돼서 tanH에 그레디언트가 곱해지게 된다.
점점 작아지게 되는 효과가 생김

<Gradient Vanishing>
Hard to train LONG sequence
길이가 길어질수록 역전파가 잘 안되는 문제 발생
그레디언트 전달이 잘 안 됨
긴 시퀀스의 데이터들은 잘 학습이 되지 않는 문제 발생
오래된 데이터들은 점점 신경쓰지 않게 됨 -> 그래서 최근에 것만 기억한다

<요약>
-여러 경로로 feed-froward될 경우, 역전파할 때
최종 그레디언트 값은 각 경로의 그레디언트들의 총 합이 된다.
(이는 다른 뉴럴네트워크에도 모두 적용되는 룰)
-RNN의 경우에도 각 타임 스텝에 따라 피드포워드되므로
각 경로로부터 전달돼 온 그레디언트들이 더해진다.
-따라서 타임 스텝의 개수만큼 레이어가 깊어진 것과 마찬가지이므로
tanh를 활성함수(논리니어 액티베이션 펑션)로 사용하는 RNN은 그레디언트 배니싱이 발생한다.
그래서 RNN은 긴 시퀀스에서는 정작 잘 동작하지 않는다.
이 문제 해결은 LSTM이라는 아키텍처를 활용해 해결 가능

## 45. Ch 08. Recurrent Neural Networks - 05. 수식 BPTT

In [ ]:
PPT, 강의 참고
<Many to One>
<Many to Many>
매니투원이 여러개 있는것처럼 미분하면 됨.
<그레디언트 배니싱>

## 46. Ch 08. Recurrent Neural Networks - 06. Long-Short Term Memory (LSTM)

In [ ]:
<그레디언트 배니싱 in Vanilla RNN>
-RNN 내부에는 tanh가 있으므로, 타임 스텝이 길어짐에 따라
그레디언트 배니싱이 발생함
	-따라서 긴 시퀀스는 학습이 어려움

<Gate using sigmoid>
-시그모이드는 0과 1사이의 값을 반환하므로
시그모이드를 곱하면 마치 문을 열고 닫는 듯한 효과를 낼 수 있음
LSTM의 기본 아이디어

<Long Short Term Memory(LSTM>
이전 타임스텝의 히든 스테이트 말고도 이전 타임스텝의 셀 스테이트를 받아 내보낸다.
셀 스테이트는 또 하나의 히든 스테이트라고 할 수 있다.
셀 스테이트는 그레디언트 소멸을 해결하는 데 중요한 역할을 한다.
 
LSTM 내부엔 게이트가 많다.
게이트의 어떤 원리에 의해 기울기 소멸 현상이 해결되는지 알아야 한다.
이러한 게이트들이 현재 타임스텝의 입력과 이전 타임스텝의 히든스테이트를 바탕으로
열지 닫을지를 결정하는 것을 배움.
그러한 게이트들이 복잡하게 얽혀서 결국 두 개(셀, 히든)의 스테이트가 각각 나오게 된다.
그렇게 해서 LSTM은 각 타임 스텝별로 진행

<Gated Recurrent Unit(GRU) [Cho et al., 2014]>
LSTM은 게이트마다 웨이트 파라미터가 다 있어서 굉장히 많고 무겁다.
그 단점을 보완하고자 하는 게 GRU
LSTM보다 훨씬 가볍다라는 장점 있음
동작 원리는 비슷

<요약>
-LSTM vs GRU?
실제로는 LSTM이 좀 더 널리 쓰이는 추세.(딱히 이유 없음)
-LSTM은 바닐라RNN에 비해서 훨씬 많은 파라미터를 가짐
따라서 더 많은 학습 데이터와 학습 시간이 필요
예전에는 LSTM이 무거워 컴퓨팅 파워 문제로 어려움이 있었지만
이제는 바닐라 RNN 쓰는 경우가 드물다.
-비록 LSTM이 그레디언트 배니싱 문제를 해결했지만,
무조건 긴 데이터를 모두 기억할 수 있는 것은 아님
	-Network cpapcity의 한계는 존재함(한정된 벡터의 크기를 가지고 있기 때문에)
	몇백 타임스텝 할 수 있다거나 그렇지는 않다는 것
	-Attention을 통해 이를 해결할 수 있음
	한 문서를 다 읽혀보고 싶을 때…등
	어텐션이 NLP를 이끌어가고 있다고 해도 과언이 아님

# 47. Ch 08. Recurrent Neural Networks - 07. Gradient Vanishing과 LSTM

In [ ]:
How does LSTM solve Gradient Vanishing?
<그레디언트 배니싱>
RNN은 BPTT라는 알고리즘을 통해서 역전파를 수행한다
BPTT는 피드포워드할 때 모든 관여된 연산들은 역전파가 동일하게 이뤄져야 한다는 것이고
그 과정에서 프사이라고 하는 웨이트 파라미터가 타임스텝마다 사용이 됐는데
그것들이 각각 역전파해서 모두 더하면 된다. 
문제는 액티베이션함수로 쓰인 tanh가 중첩된 상태로
역전파 과정에서 이것이 계속해서 곱해지게 되면 기울기 소멸 현상이 발생
뒤쪽에 있는 타임스텝의 프사이는 큰 영향을 받게 되지만
앞쪽에 있는 타임스텝의 프사이는 굉장히 미미한 영향만 받게 됨.
프사이가 바뀌지 않음
이는 마치 Long range distance를 기억하지 못한다는 것과 같게 됨
바닐라 RNN은 롱시퀀스를 잘 학습하지 못하고 앞쪽에 있는 것을 잘 기억하지 못한다는 문제가 생김

<BPTT in LSTM>
LSTM에서는 BPTT가 일어날 때 기울기 소멸 문제가 발생하지 않도록 하는 장치들이 있다
LSTM은 히든 스테이트와, 셀 스테이트라는 또 하나의 히든 스테이트가 존재.
셀 스테이트는 현재 자기 자신의 셀 스테이트를 만드는 데 있어서 지난 타임 스텝의 셀 스테이트가 들어감
f를 어떻게 배우냐에 따라서 그레디언트를 뒤로 전달하거냐 말거냐가 정해진다

<Still we have gradient vanishing in depth>
LSTM solves only in time axis.
레이어도 너무 깊게 쌓으면 기울기 소멸 문제 발생
깊게 쌓고 싶으면 나중에 배울 residual connection 등을 이용해 깊게 쌓을 수 있다
그러면 시간축 뿐만 아니라 뎁스에 대해서도 그레디언트 배니싱을 해결할 수 있다
레이어는 4개 정도 권장. 초과해서 쌓으면 무의미한 것 같다
구글에서 출판한 시퀀스투시퀀스 기계번역 같은 경우에는 4개까지는 괜찮고
그보다 많이 쌓을 때는 residual connection을 쓰는 게 훨씬 낳다
자기들도 그걸 사용해서 8개 층을 쌓았다라고 밝힘.
트랜스포머 이전의 시퀀스투시퀀스 썼을 때 기계번역 얘기.

# TODO : 48. Ch 08. Recurrent Neural Networks - 08. 실습 브리핑